# PHASE 2 PROJECT

Business Objective:
Your company is launching a movie studio. You have been tasked with analyzing current trends in movies to provide 3 actionable recommendations for what types of movies the studio should produce.

Key Business Questions:

What genres tend to perform best at the box office?

Are highly rated movies (IMDB ratings) more successful?

Does release timing (month) influence box office performance?

2. Data Loading and Initial Exploration
Import Libraries

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import pandas as pd

Load Datasets

In [9]:
conn = sqlite3.connect('im.db')

# Check available tables
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Load movie_basics and movie_ratings
df_basics = pd.read_sql_query("SELECT * FROM movie_basics;", conn)
df_ratings = pd.read_sql_query("SELECT * FROM movie_ratings;", conn)